#augument the data

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
from os import listdir
import time   

In [2]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m}:{round(s,1)}"

In [3]:
def augment_data(file_dir, n_generated_samples, save_to_dir):
    data_gen = ImageDataGenerator(rotation_range=30, 
                                  width_shift_range=0.1,
                                  height_shift_range=0.15, 
                                  shear_range=0.25, 
                                  zoom_range = 0.2,
                                  horizontal_flip=True, 
                                  vertical_flip=False, 
                                  fill_mode='nearest',
                                  brightness_range=(0.5,1.2)
                                 )

    
    for filename in listdir(file_dir):
        image = cv2.imread(file_dir + '/' + filename)
        image = image.reshape((1,)+image.shape)
        save_prefix = 'aug_' + filename[:-4]    
        i=0
        for batch in data_gen.flow(x=image, batch_size=1, save_to_dir=save_to_dir,
                                   save_prefix=save_prefix, save_format='jpg'):
            i += 1
            if i > n_generated_samples:
                break

In [5]:
start_time = time.time()

augmented_data_path = '/content/drive/MyDrive/Digital Naturalist Dataset'
augment_data(file_dir='/content/drive/MyDrive/Digital Naturalist Dataset/Bird/Great Indian Bustard Bird', n_generated_samples=8, save_to_dir=augmented_data_path+'/Bird/Great Indian Bustard Bird')
augment_data(file_dir='/content/drive/MyDrive/Digital Naturalist Dataset/Bird/Spoon Billed Sandpiper Bird', n_generated_samples=8, save_to_dir=augmented_data_path+'/Bird/Spoon Billed Sandpiper Bird')
augment_data(file_dir='/content/drive/MyDrive/Digital Naturalist Dataset/Flower/Corpse Flower', n_generated_samples=8, save_to_dir=augmented_data_path+'/Flower/Corpse Flower')
augment_data(file_dir='/content/drive/MyDrive/Digital Naturalist Dataset/Flower/Lady Slipper Orchid Flower', n_generated_samples=8, save_to_dir=augmented_data_path+'/Flower/Lady Slipper Orchid Flower')
augment_data(file_dir='/content/drive/MyDrive/Digital Naturalist Dataset/Mammal/Pangolin Mammal', n_generated_samples=8, save_to_dir=augmented_data_path+'/Mammal/Pangolin Mammal')
augment_data(file_dir='/content/drive/MyDrive/Digital Naturalist Dataset/Mammal/Senenca White Deer Mammal', n_generated_samples=8, save_to_dir=augmented_data_path+'/Mammal/Senenca White Deer Mammal')


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
end_time = time.time()
execution_time = (end_time - start_time)
print("Elapsed Time : "+str(execution_time))

Elapsed Time : 85.99580574035645


#loading data and preprocessing

In [7]:
#Importing Libraries

#Locating and loading datasets
import pathlib
from pathlib import Path
import os, gc, glob, random
from PIL import Image

#DataManagement and matrix calculations
import pandas as pd
import numpy as np

#Model Building
import tensorflow as tf
import keras 
import keras.backend as K
from keras.optimizers import SGD, Adam, Adagrad, RMSprop
from keras.applications import *
from keras.preprocessing import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, BatchNormalization,Dropout
from keras.models import Model
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

# Data Visualization
import matplotlib.pyplot as plt

#Loading and testing models
from keras.models import load_model
from keras.models import model_from_json

# Directory operations
import os
from os import listdir

#define required function

In [8]:
def generateListofFiles(dirName):
    """This function returns a list with exact paths of files inside the given directory """
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for fol_name in listOfFile:
        fullPath = os.path.join(dirName, fol_name)
        allFiles.append(fullPath)
              
    return allFiles

In [9]:
def Configure_CNN_Model(output_size):
    """This function defines the cnn model structure and configures the layers"""
    K.clear_session()
    model = Sequential()
    model.add(Dropout(0.4,input_shape=(224, 224, 3)))
    
    model.add(Conv2D(256, (5, 5),input_shape=(224, 224, 3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    #model.add(BatchNormalization())

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    #model.add(BatchNormalization())

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    #model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(output_size, activation='softmax'))
    
    return model

In [10]:
def PrepreocessData(subfolders):
    """Pre precess the image data in the provided category list"""
    X_data,Y_data,found = [],[],[]
    id_no=0
    #itering in all folders under Boats folder
    for paths in subfolders:
        #setting folder path for each boat type
        files = glob.glob (paths + "/*.jpg")
        found.append((paths.split('/')[-2],paths.split('/')[-1]))
        
        #itering all files under the folder one by one
        for myFile in files:
            img = Image.open(myFile)
            #img.thumbnail((width, height), Image.ANTIALIAS) # resizes image in-place keeps ratio
            img = img.resize((224,224), Image.ANTIALIAS) # resizes image without ratio
            #convert the images to numpy arrays
            img = np.array(img)
            if img.shape == ( 224, 224, 3):
                # Add the numpy image to matrix with all data
                X_data.append (img)
                Y_data.append (id_no)
        id_no+=1

    #converting lists to np arrays again
    X = np.array(X_data)
    Y = np.array(Y_data)

    # Print shapes to see if they are correct
    print("x-shape",X.shape,"y shape", Y.shape)

    X = X.astype('float32')/255.0
    y_Bird = to_categorical(Y_data, len(subfolders))

    print("X shape",X,"y_Bird shape", y_Bird)
    print("X shape",X.shape,"y_Bird shape", y_Bird.shape)

    return X_data,Y_data,X,y_Bird,found; 

In [11]:
def splitData():
    X_train, X_test, y_train, y_test = train_test_split(X, y_Bird, test_size=0.2)
    print("The model has " + str(len(X_train)) + " inputs")
    return X_train, X_test, y_train, y_test

#The AugumentData.py will generate many images with the original dataset to increase the accuracy of the model.

#BUILDING THE CNN MODEL

In [15]:
early_stop_loss = EarlyStopping(monitor='loss', patience=3, verbose=1)
early_stop_val_acc = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
model_callbacks=[early_stop_loss, early_stop_val_acc]

In [16]:
model = Configure_CNN_Model(6)
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
weights = model.get_weights()
model.set_weights(weights)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


#PREDECTING IMAGE CLASSES

In [17]:
# random.randint(0,len(X_test))
# [X_test[image_number]

#SAVING THE MODEL LOCALLY

In [19]:
model_json = model.to_json() #indent=2
with open("DigitalNaturalist.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to H5

model.save_weights("DigitalNaturalist.h5")
print("Saved model to disk")

Saved model to disk


In [20]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-11-19 04:45:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  42.4MB/s    in 0.3s    

2022-11-19 04:45:31 (42.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [21]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [22]:
get_ipython().system_raw('./ngrok http 8501 &')

In [23]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://1e47-34-173-132-230.ngrok.io


In [24]:
!streamlit run /content/predict_image_class.py

/bin/bash: streamlit: command not found


#next method

In [25]:
!unzip "/content/data.rar"

unzip:  cannot find or open /content/data.rar, /content/data.rar.zip or /content/data.rar.ZIP.


In [26]:
import os, gc, glob, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from PIL import Image
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [27]:
base_model = InceptionV3(input_shape=(224,224,3),include_top=False)

87910968/87910968 [==============================] - 1s 0us/step


In [28]:
for layer in base_model.layers:
  layer.trainable = False

In [29]:
X = Flatten()(base_model.output)

#using sigmoid function as activation funciton
X = Dense(units = 6, activation = 'sigmoid')(X)

In [30]:
model = Model(base_model.input,X)

#using adam as the optimizer
#accuracy is set as the metrics to judge the performance of the model
model.compile(optimizer = 'adam',loss = keras.losses.binary_crossentropy,metrics=['accuracy'])

#summarizing the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d_3[0][0]']               
 alization)                     )                                                             

In [31]:
#initializing the datagenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

#loading the train dataset from the directory
traindata= datagen.flow_from_directory(directory="/content/drive/MyDrive/Augumented dataset/Augumented Dataset",target_size=(224,224),batch_size=100)

#loading the test dataset from the directory
testdata= datagen.flow_from_directory(directory="/content/drive/MyDrive/Augumented dataset/Augumented Dataset",target_size=(224,224),batch_size=100)

Found 2050 images belonging to 6 classes.
Found 2050 images belonging to 6 classes.


In [32]:
#listing the found classes from train dataset
traindata.class_indices

{'Corpse Flower': 0,
 'Great Indian Bustard Bird': 1,
 'Lady Slipper Orchid Flower': 2,
 'Pangolin Mammal': 3,
 'Senenca White Deer Mammal': 4,
 'Spoon Billed Sandpiper Bird': 5}